In [118]:
import pandas as pd

journeys = pd.read_csv("processed_journey/test_output.csv.gz", compression='gzip')

In [125]:
journeys.head(10)

,Sequence,PageSequence,Occurrences,Page_Seq_Occurrences,DeviceCategories,Dates,Page_Event_List,Page_List,PageSequence_internal,Event_List,num_event_cats,Event_cats_agg,Event_cat_act_agg,Page_List_NL,Page_Seq_NL,Occurrences_NL,contains_search,count_search,contains_search_n
0,/guidance/national-clearance-hub-for-goods-ent...,/guidance/national-clearance-hub-for-goods-ent...,33,34.0,"[('desktop', 26), ('mobile', 6), ('tablet', 1)]","[('20181018', 33)]",[('/guidance/national-clearance-hub-for-goods-...,['/guidance/national-clearance-hub-for-goods-e...,/guidance/national-clearance-hub-for-goods-ent...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/guidance/national-clearance-hub-for-goods-e...,/guidance/national-clearance-hub-for-goods-ent...,68.0,0,0,0.0
1,/government/publications/ministry-of-defence-p...,/government/publications/ministry-of-defence-p...,33,37.0,"[('desktop', 26), ('mobile', 7)]","[('20181018', 33)]",[('/government/publications/ministry-of-defenc...,['/government/publications/ministry-of-defence...,/government/publications/ministry-of-defence-p...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/publications/ministry-of-defence...,/government/publications/ministry-of-defence-p...,187.0,0,0,0.0
2,/vehicle-registration/new-registrations-fee<<P...,/vehicle-registration/new-registrations-fee,35,39.0,"[('mobile', 19), ('desktop', 12), ('tablet', 4)]","[('20181018', 35)]",[('/vehicle-registration/new-registrations-fee...,['/vehicle-registration/new-registrations-fee'],/vehicle-registration/new-registrations-fee,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/vehicle-registration/new-registrations-fee'],/vehicle-registration/new-registrations-fee,197.0,0,0,0.0
3,/tax-sell-property<<PAGE<:<NULL<:<NULL>>/tax-s...,/tax-sell-property>>/tax-sell-property/work-ou...,34,95.0,"[('mobile', 13), ('tablet', 2), ('desktop', 19)]","[('20181018', 34)]","[('/tax-sell-property', 'PAGE<:<NULL<:<NULL'),...","['/tax-sell-property', '/tax-sell-property/wor...",/tax-sell-property>>/tax-sell-property/work-ou...,"[('PAGE_NULL', 'PAGE_NULL'), ('contentsClicked...",3,"[('PAGE_NULL', 2), ('contentsClicked', 1), ('E...","[(('PAGE_NULL', 'PAGE_NULL'), 2), (('contentsC...","['/tax-sell-property', '/tax-sell-property/wor...",/tax-sell-property>>/tax-sell-property/work-ou...,2388.0,0,0,0.0
4,/guidance/river-thames-bridges-locks-and-facil...,/guidance/river-thames-bridges-locks-and-facil...,36,47.0,"[('mobile', 26), ('tablet', 5), ('desktop', 5)]","[('20181018', 36)]",[('/guidance/river-thames-bridges-locks-and-fa...,['/guidance/river-thames-bridges-locks-and-fac...,/guidance/river-thames-bridges-locks-and-facil...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/guidance/river-thames-bridges-locks-and-fac...,/guidance/river-thames-bridges-locks-and-facil...,485.0,0,0,0.0
5,/guidance/spirits-duty<<PAGE<:<NULL<:<NULL,/guidance/spirits-duty,35,46.0,"[('tablet', 1), ('desktop', 12), ('mobile', 22)]","[('20181018', 35)]","[('/guidance/spirits-duty', 'PAGE<:<NULL<:<NUL...",['/guidance/spirits-duty'],/guidance/spirits-duty,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/guidance/spirits-duty'],/guidance/spirits-duty,509.0,0,0,0.0
6,/private-renting/repairs<<PAGE<:<NULL<:<NULL,/private-renting/repairs,106,122.0,"[('mobile', 72), ('desktop', 27), ('tablet', 7)]","[('20181018', 106)]","[('/private-renting/repairs', 'PAGE<:<NULL<:<N...",['/private-renting/repairs'],/private-renting/repairs,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/private-renting/repairs'],/private-renting/repairs,1274.0,0,0,0.0
7,/guidance/eu-settlement-scheme-pilot-applicant...,/guidance/eu-settlement-scheme-pilot-applicant...,477,546.0,"[('desktop', 246), ('mobile', 205), ('tablet',...","[('20181018', 477)]",[('

In [120]:
def contains_search(Page_List):
    search_in_sequence = 0
    thelist  = pd.eval(Page_List)
    
    for page in thelist:
        
        if ('/search?' in page) or ('/search/' in page):
            search_in_sequence = 1
            print(page)
    return search_in_sequence

def count_search(Page_List):
    times_search_in_sequence = 0
    thelist  = pd.eval(Page_List)
    
    for page in thelist:
        if ('/search?' in page) or ('/search/' in page):
            times_search_in_sequence += 1
            print(page)
    return times_search_in_sequence


journeys['contains_search'] = list(map(contains_search, journeys['Page_List']))

journeys['count_search'] = list(map(count_search, journeys['Page_List']))

/search?q=universal+credit+login
/search?q=xx
/search?q=newly+passed+hgv+
/search?q=head+of+london+nps&show_organisations_filter=true
/search?q=tax+identification+number
/search?q=moving+home
/search?q=log+in
/search?q=[postcode]
/search?q=utr
/search?q=universal+credit
/search?q=universal+credit
/search?q=nino
/search?q=log+in+universal+credit
/search?q=c100
/search?q=sorn
/search?q=change+name
/search?q=universal+credit+
/search?q=universal
/search?q=[postcode]fund
/search?q=find+an+apprenticeship
/search?q=life+in+uk
/search?q=p800
/search?q=gro
/search?q=pension
/search?q=childcare+account
/search?q=apprenticship
/search?q=theory+test
/search?q=dbs
/search?q=prison
/search?q=tax+free+childcare
/search?q=register+to+vote+
/search?q=test
/search?q=divorce
/search?q=working+tax+credits
/search?q=national+insurance+number+
/search?q=gabapentin+
/search?q=how+to+rent
/search?q=30+hours
/search?q=childcare+account
/search?q=book+driving+test
/search?q=civil+service+jobs
/search?q=driving

/search?q=sscs1+form
/search?q=ey2
/search?q=find+a+job
/search?q=apply+for+national+insurance+number
/search?q=sic+codes
/search?q=chv1
/search?q=update+name
/search?q=driving+test
/search?q=childcare+account
/search?q=form+64-8
/search?q=p11d
/search?q=n180
/search?q=n181&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=n180&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=check+if+taxed
/search?q=manage+tax+credit
/search?q=test
/search?q=64-8
/search?q=universal+credits
/search?q=vehicle+tax
/search?q=manage
/search?q=universal+credit
/search?q=pta
/search?q=help+to+save
/search?q=vehicle+tax
/search?q=mot+check
/search?q=wills
/search?q=tax+free+childcare
/search?q=tax+free+child+care
/search?q=passport+renewal
/search?q=check+car+tax
/search?q=apply+visa
/search?q=64/8
/search?q=sign+in
/search?q=success+profiles
/search?q=employment+status
/search?q=universal+credit+login
/search?q=power+of+attorney+form
/search?q=child+care+account
/search?q=tax+

/search?q=childcare+account
/search?q=universal+credit
/search?q=test
/search?q=universal+credit
/search?q=cis
/search?q=car+tax
/search?q=sorn
/search?q=maternity+grant
/search?q=universal+credit
/search?q=iht205
/search?q=universal+credit
/search?q=register+to+vote
/search?q=oc1
/search?q=check+code
/search?q=personal+tax+account
/search?q=login+
/search?q=dbs
/search?q=[postcode]fund
/search?q=esta
/search?q=esta
/search?q=login
/search?q=pension
/search?q=universal+credit
/search?q=universal+credit
/search?q=universal+credit
/search?q=universal+credit
/search?q=universal+credit
/search?q=maternity+grant
/search?q=d81&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=d81&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=sign+in
/search?q=sign+in
/search?q=carers+allowance
/search?q=student+finance+forms
/search?q=sold
/search?q=state+pension
/search?q=personal+tax+account
/search?q=hmrc+services
/search?q=hmrc+services
/search?q=student+finance
/search?

/search?o=validate+identifacation&q=validate+identification
/search?q=passport
/search?q=newly+passed+hgv+
/search?q=pension+
/search?q=theory+test
/search?q=student+finance+forms
/search?q=childcare
/search?q=apply+national+insurance+number+card
/search?q=d80b&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=book+driving+test
/search?q=universal+credit
/search?q=entry+requirements
/search?q=
/search?q=driving+license+
/search?q=claim+esa
/search?q=driving+test+
/search?q=check+tax
/search?q=childcare+account
/search?q=log+in+
/search?q=p800+refund
/search?q=student+finance
/search?q=login
/search?q=childcare+account+
/search?q=national+insurance+
/search?q=mot+
/search?q=hmrc
/search?q=driving+
/search?q=vehicle+tax
/search?q=ch2
/search?q=register+to+vote
/search?q=find+a+apprenticeship+
/search?q=book+driving+test
/search?q=driving+license+
/search?q=rhi
/search?q=is+my+car+taxed
/search?q=car
/search?q=private+plate
/search?q=childcare+account
/search?q=manage+my+ta

/search?q=carers
/search?q=mot+histor
/search?q=mot+
/search?q=p87
/search?q=check+mot
/search?q=child+care+account
/search?q=apply+for+british+passport
/search?q=sign+in
/search?q=tax
/search?q=childcare+account
/search?q=universal+credit
/search?q=starter+checklist
/search?q=bank+holiday
/search?q=childcare+account
/search?q=apply+for+british+passport
/search?q=mot+history
/search?q=[postcode]
/search?q=dvla
/search?q=manage+my+tax+credits
/search?q=tax+free+childcare
/search?q=benefit+calculator
/search?q=ex107&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=childcare+account
/search?q=ex107&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=permanent+residency
/search?q=[postcode]
/search?q=rdr3
/search?q=nhs
/search?q=universal+credit+account
/search?q=marriage+allowance
/search?q=form+n244&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=check+code
/search?q=life+in+uk
/search?q=sa103
/search?q=log+in
/search?q=power+of+attorney
/sea

...

/search?q=passport
/search?q=universal+credit
/search?q=theory
/search?q=p87
/search?q=p85
/search?q=school+workforce+census
/search?q=ccj
/search?q=tax+my+vehicle
/search?q=intrastat
/search?q=sign+into+universal+credit
/search?q=pension+forecast
/search?q=esa
/search?q=p800
/search?q=childcare+account
/search?q=sold
/search?q=contact
/search?q=smp
/search?q=pension+credit
/search?q=vat
/search?q=lrs
/search?q=dbs+update
/search?q=companies+house
/search?q=electrician
/search?q=road+tax
/search?q=c100
/search?q=help+to+save
/search?q=personal+independence+payment
/search?q=lost+utr
/search?q=mot
/search?q=pension
/search?q=cis
/search?q=emergency+travel+document
/search?q=universal+credit
/search?q=uniform+tax
/search?q=practice+guides
/search?q=change+of+address
/search?q=childcare+account
/search?q=register+for+self+assessment
/search?q=apprenticeships
/search?q=universal+credit
/search?q=universal+credit
/search?q=national+insurance+contributions
/search?q=tier+2+and+5+guidance
/se

/search?q=universal+credit
/search?q=mot+
/search?q=universal+credit
/search?q=p45
/search?q=personal+tax+account
/search?q=mot
/search?q=personal+tax+account
/search?q=visa
/search?q=visa&start=20
/search?q=visa&start=40
/search?q=universal+credit
/search?q=mot
/search?q=paye+login
/search?q=crmr1
/search?q=hmrc+services
/search?q=exchange+rates
/search?q=universal+credit
/search?q=trust+fund
/search?q=universal+credit+login+
/search?q=[postcode]fund
/search?q=universal+credit
/search?q=car+tax
/search?q=universal+credit
/search?q=personal+tax+account
/search?q=mot
/search?q=uniform
/search?q=check+vehicle+tax
/search?q=exchange+rates
/search?q=universal+credit
/search?q=apprenticeship+
/search?q=self+assessment
/search?q=hmrc
/search?q=sa1
/search?q=vehicle+tax
/search?q=universal+credit
/search?q=continuation+sheet
/search?q=p87
/search?q=sign+in
/search?q=universal+credit
/search?q=universal+credit+
/search?q=dbs
/search?q=apprenticeships
/search?q=uc50
/search?q=universal+credit
/

/search?q=benefits
/search?q=p87
/search?q=immigration+adviser
/search?q=car+tax
/search?q=vat
/search?q=cwf1
/search?q=blue+badge
/search?q=oc2
/search?q=cpp16
/search?q=mot+history
/search?q=apprenticeship
/search?q=n510&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=probate
/search?q=mot
/search?q=dbs+check
/search?q=c110a&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=ex343&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=hmrc
/search?q=precedent+h&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=parentview
/search?o=parentview&q=parent+view
/search?q=kerch
/search?q=brexit
/search?q=certificate+of+service&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=statistics
/search?q=tax+overview
/search?q=mot+check
/search?q=childcare+account
/search?q=tier+2+visa
/search?q=self+employed
/search?q=data+protection+act+1998
/search?q=log+in
/search?q=register+to+vote
/search?q=goverment+gateway
/search?q=last

/search?q=self+assessment
/search?q=64-8
/search?q=passport
/search?q=sign+in
/search?q=sign+in
/search?q=employment+tribunal+
/search?q=sold+car
/search?q=ehic
/search?q=742a
/search?q=742a
/search?q=utr
/search?q=universal+credit
/search?q=universal+credit
/search?q=dla
/search?q=universal+credit
/search?q=paying+hmrc
/search?q=universal+credit
/search?q=tax+free+childcare
/search?q=p800
/search?q=budgeting+loan
/search?q=universal+credit
/search?q=universal+credit
/search?q=tax+credits
/search?q=ex107&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=universal+credit
/search?q=universal+credit
/search?q=universal+credit
/search?q=pensions
/search?q=universal+credit
/search?q=v5
/search?q=road+tax
/search?q=ch2
/search?q=passport+renewal
/search?q=sim
/search?q=sim
/search?q=p55
/search?q=p55
/search?q=universal+credit
/search?q=self+assesment
/search?q=universal+credit
/search?q=budgeting+loan
/search?q=ch2
/search?q=masters+loan
/search?q=professional+subscriptions
/

/search?q=overseas+passport
/search?q=n1&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=universal+credit+
/search?q=personal+tax+account+
/search?q=budgeting+loan
/search?q=apprenticeship
/search?q=c100&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=universal+credit
/search?q=visa
/search?q=universal+credit+
/search?q=personal+tax+account
/search?q=universal
/search?q=p87
/search?q=road+tax
/search?q=64-8
/search?q=cis
/search?q=universal+credit
/search?q=dart+charge
/search?q=universal+credit
/search?q=universal+credit
/search?q=universal+credit
/search?q=
/search?q=vat
/search?q=car+tax
/search?q=universal+credit+
/search?q=car+tax
/search?q=apprenticeships


In [121]:
journeys['contains_search_n'] = journeys['contains_search']*journeys['Page_Seq_Occurrences']
journeys['contains_search_n']



0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8         64.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14         0.0
15         0.0
16         0.0
17         0.0
18         0.0
19         0.0
20         0.0
21         0.0
22         0.0
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
28         0.0
29         2.0
          ... 
606705     0.0
606706     0.0
606707     0.0
606708     0.0
606709     0.0
606710     0.0
606711     0.0
606712     0.0
606713     0.0
606714     0.0
606715     0.0
606716     0.0
606717     0.0
606718     0.0
606719     0.0
606720     0.0
606721     0.0
606722     0.0
606723     0.0
606724     0.0
606725     0.0
606726     0.0
606727     0.0
606728     0.0
606729     0.0
606730     0.0
606731     0.0
606732     0.0
606733     0.0
606734     0.0
Name: contains_search_n, Length: 606735, dtype: float64

In [124]:
print("{}/{} ({}%) of journeys contain internal search in their path".format(journeys['contains_search_n'].sum(),
                                                                             journeys['Page_Seq_Occurrences'].sum(),
                                                                             journeys['contains_search_n'].sum()/journeys['Page_Seq_Occurrences'].sum()*100))

298518.0/6389956534.0 (0.004671674970113341%) of journeys contain internal search in their path


In [117]:
journeys['contains_search'].value_counts()

0    597298
1      9437
Name: contains_search, dtype: int64

In [98]:
journeys['contains_search'].value_counts(normalize=True)

0    0.984446
1    0.015554
Name: contains_search, dtype: float64

In [104]:
journeys['count_search'].describe()

count    606735.000000
mean          0.016203
std           0.131750
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           4.000000
Name: count_search, dtype: float64

In [ ]:
def df['length'] = len

In [28]:
df1 = df[df['Occurrences']<5]

In [33]:
df1

,Sequence,PageSequence,Occurrences,Page_Seq_Occurrences,DeviceCategories,Dates,Page_Event_List,Page_List,PageSequence_internal,Event_List,num_event_cats,Event_cats_agg,Event_cat_act_agg,Page_List_NL,Page_Seq_NL,Occurrences_NL
25,/government/organisations/nuclear-decommission...,/government/organisations/nuclear-decommission...,1,3.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/organisations/nuclear-decommiss...,['/government/organisations/nuclear-decommissi...,/government/organisations/nuclear-decommission...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/organisations/nuclear-decommissi...,/government/organisations/nuclear-decommission...,9.0
26,/foreign-travel-advice/new-zealand/travel-advi...,/foreign-travel-advice/new-zealand/travel-advi...,1,2.0,"[('mobile', 1)]","[('20181018', 1)]",[('/foreign-travel-advice/new-zealand/travel-a...,['/foreign-travel-advice/new-zealand/travel-ad...,/foreign-travel-advice/new-zealand/travel-advi...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/foreign-travel-advice/new-zealand/travel-ad...,/foreign-travel-advice/new-zealand/travel-advi...,4.0
27,/government/publications/italy-list-of-transla...,/government/publications/italy-list-of-transla...,1,3.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/publications/italy-list-of-tran...,['/government/publications/italy-list-of-trans...,/government/publications/italy-list-of-transla...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/publications/italy-list-of-trans...,/government/publications/italy-list-of-transla...,9.0
28,/employment-tribunal-decisions/mrs-p-bhatia-an...,/employment-tribunal-decisions/mrs-p-bhatia-an...,1,2.0,"[('mobile', 1)]","[('20181018', 1)]",[('/employment-tribunal-decisions/mrs-p-bhatia...,['/employment-tribunal-decisions/mrs-p-bhatia-...,/employment-tribunal-decisions/mrs-p-bhatia-an...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/employment-tribunal-decisions/mrs-p-bhatia-...,/employment-tribunal-decisions/mrs-p-bhatia-an...,4.0
29,/search?q=xx<<PAGE<:<NULL<:<NULL,/search?q=xx,1,2.0,"[('mobile', 1)]","[('20181018', 1)]","[('/search?q=xx', 'PAGE<:<NULL<:<NULL')]",['/search?q=xx'],/search?q=xx,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/search?q=xx'],/search?q=xx,4.0
30,/government/publications/teacher-misconduct-pa...,/government/publications/teacher-misconduct-pa...,1,2.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/publications/teacher-misconduct...,['/government/publications/teacher-misconduct-...,/government/publications/teacher-misconduct-pa...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/publications/teacher-misconduct-...,/government/publications/teacher-misconduct-pa...,4.0
31,/government/publications/hungary-consular-fees...,/government/publications/hungary-consular-fees,1,2.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/publications/hungary-consular-f...,['/government/publications/hungary-consular-fe...,/government/publications/hungary-consular-fees,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/publications/hungary-consular-fe...,/government/publications/hungary-consular-fees,4.0
32,/government/publications/childrens-commissione...,/government/publications/childrens-commissione...,1,3.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/publications/childrens-commissi...,['/government/publications/childrens-commissio...,/government/publications/childrens-commissione...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/publications/childrens-commissio...,/government/publications/childrens-commissione...,6.0
33,/government/publications/denaturing-of-control...,/go

In [38]:
df1['Sequence'][602379]

'/government/organisations/companies-house<<PAGE<:<NULL<:<NULL>>/government/organisations/companies-house<<EVENT<:<External Link Clicked<:<https://beta.companieshouse.gov.uk/>>/government/organisations/companies-house<<PAGE<:<NULL<:<NULL>>/file-an-annual-return-with-companies-house<<PAGE<:<NULL<:<NULL>>/file-an-annual-return-with-companies-house<<EVENT<:<External Link Clicked<:<https://ewf.companieshouse.gov.uk/>>/government/organisations/companies-house<<PAGE<:<NULL<:<NULL>>/government/organisations/companies-house<<EVENT<:<External Link Clicked<:<https://beta.companieshouse.gov.uk/'